In [10]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import json
import pathlib

In [ ]:
client = Elasticsearch(hosts='http://localhost:9008')
search = Search(using=client, index='documents')

In [12]:
document_dump = {}
for hit in search.scan():
    document_dump[hit.id] = hit.to_dict()
with open('document_dump.json', 'w', encoding='utf-8') as f:
    json.dump(document_dump, f, ensure_ascii=False, indent=2)

In [ ]:
client.indices.create(
    index='documents_stem',

)